In [9]:
import os
!pip install pygame

In [10]:
import numpy as np
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [11]:
pygame.init()

(5, 0)

In [12]:
class conways:
    def __init__(self, size = 100, max_it = 100):
        self.max_it = max_it
        screen_size = (1024, 1024)
        self.liveCellColor = (255, 255, 255)
        self.deadCellColor = (25, 25, 25)
        self.screen = pygame.display.set_mode(screen_size)

        self.x_size = size+2
        self.y_size = size+2
        self.x_bound = self.x_size - 1
        self.y_bound = self.y_size - 1

        self.square_off = int(screen_size[0]/(self.x_size-2))

        self.buff_1 = np.random.choice(
            a = [False, False, False, False, False, True, True], 
            size = (self.x_size, self.y_size))
        self.buff_2 = np.copy(self.buff_1)

        self.buff = []
        self.buff.append(self.buff_1)
        self.buff.append(self.buff_2)

        self.reset(self.buff_1)

    def reset(self, a):
        self.moves = 0
        self.buff_1 = a

        self.buff[0] = self.buff_1
        self.buff[1] = self.buff_2
        for y in range(1, self.y_bound):
            for x in range(1, self.x_bound):
                if self.buff_1[y, x]:
                    self.moves += 1
        #reset padding
        for x in range(self.x_size):
            self.buff_1[0, x] = False
            self.buff_1[x, 0] = False
            self.buff_1[self.y_bound, x] = False
            self.buff_1[x, self.y_bound] = False


        self.buff_2 = np.copy(self.buff_1)


    def run(self):
        for i in range(self.max_it):
            changed = False
            self.screen.fill(self.deadCellColor)

            for y in range(1, self.y_bound):
                for x in range(1, self.x_bound):
                    self.buff[1][y, x] = False
                    count = 0

                    if (self.buff[0][y - 1, x]):
                        count += 1
                    if (self.buff[0][y - 1, x - 1]):
                        count += 1
                    if (self.buff[0][y - 1, x + 1]):
                        count += 1
                    if (self.buff[0][y, x - 1]):
                        count += 1
                    if (self.buff[0][y + 1, x - 1]):
                        count += 1
                    if (self.buff[0][y + 1, x + 1]):
                        count += 1
                    if (self.buff[0][y + 1, x]):
                        count += 1
                    if (self.buff[0][y, x + 1]):
                        count += 1

                    if (not(self.buff[0][y, x]) and 
                    count == 3):
                        self.buff[1][y, x] = True
                        changed = True
                        
                    if (self.buff[0][y, x] and count < 2 or count > 3):
                        self.buff[1][y, x] = False
                        changed = True
                    if self.buff[0][y, x]:
                        self.screen.fill(
                            self.liveCellColor, 
                            rect=(self.square_off*(y-1), self.square_off*(x-1), 
                            self.square_off, self.square_off))



            
            pygame.display.flip()
            #time.sleep(1)
            if (self.buff[0] is self.buff_1):
                self.buff[0] = self.buff_2
                self.buff[1] = self.buff_1
            else:
                self.buff[0] = self.buff_1
                self.buff[1] = self.buff_2


            if not(changed):
                alive = False
                #print(buff[0])
                for y in range(1, self.y_bound):
                    for x in range(1, self.x_bound):
                        if self.buff[0][y, x]:
                            alive = True
                            break
                    if alive:
                        break
                if alive:
                    return float('inf')
                else:
                    return i**2 - self.moves
                
        actions = self.max_it**2 - self.moves  
        return self.buff, actions

In [38]:
env = conways(size = 512, max_it=1000)
print(env.run())

-74903


In [39]:
x = np.zeros(shape=(514, 514), dtype=bool)
x[100, 100] = True
x[100, 101] = True
x[101, 100] = True
x[101, 101] = False


In [40]:
env.reset(x)
actions = env.run()



In [41]:
print(actions)

6


In [42]:
alpha = 0.1 
max_iterations = 100000
gamma = 0.995

total = 0
tot_reward = 0
prev_reward = 0

observation = [30, 30, 50, 50]
array_win = np.array([0.25, 0.25, 0.01, 0.1])

epsilon = 1
epsilon_decay = 0.99995

In [43]:
q_table = np.random.uniform(low=0, high=1, size=(observation + [actions]))
q_table.shape

(30, 30, 50, 50, 6)

In [46]:
magic_array =  np.array([15,10,1,10])
def get_discrete_state(state):
    discrete_state = state/array_win + magic_array
    return tuple(discrete_state.astype(int))

In [50]:
for i in range(max_iterations):
    t0 = time.time()
    discrete_state = env.reset(x)
    done = False

    reward = 0

    if (i % 2000 == 0):
        print(f"Episode: {i}")
    
    while(not(done)):
        
        if np.random.random() > epsilon:
            action = np.argmax(q_table[discrete_state])
        else:
            action = np.random.randint(0, actions)

        new_state = actions

        reward += actions

        new_discrete_state = new_state

        # if (i % 2000 == 0):
        #     env.render()
        
        
        if not done:
            #print(q_table[discrete_state])
            
            max_future_q = np.max(q_table[new_discrete_state])
            current_q = q_table[discrete_state + action]
            
            new_q = (1 - alpha) * current_q + alpha * (reward + gamma * max_future_q)

            q_table[discrete_state + (action,)] = new_q
        
        discrete_state = new_discrete_state

    if epsilon > 0.05: 
        if reward > prev_reward and i > 10000:
            epsilon = math.pow(epsilon_decay, i - 10000)

            if i % 500 == 0:
                print("Epsilon: " + str(epsilon))
    
    t1 = time.time()
    iteration_time = t1-t0
    total += iteration_time

    tot_reward += reward
    prev_reward = reward

    if i % 1000 == 0: 
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = tot_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        tot_reward = 0


Episode: 0


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
env.close()

In [ ]:
print(q_table.shape)

In [ ]:
print(q_table)

In [ ]:
for i in range(max_iterations):

    discrete_state = get_discrete_state(env.reset())
    done = False

    reward = 0

    
    while(not(done)):
        
        action = np.argmax(q_table[discrete_state])


        new_state, step_reward, done, info = env.step(action)

        reward += step_reward

        new_discrete_state = get_discrete_state(new_state)

        env.render()
        
        if not done:
            max_future_q = np.max(q_table[new_discrete_state])
            current_q = q_table[discrete_state + (action,)]
            
            new_q = (1 - alpha) * current_q + alpha * (reward + gamma * max_future_q)

            q_table[discrete_state + (action,)] = new_q
        
        discrete_state = new_discrete_state